In [2]:
import os
import json
from IPython.display import display, Markdown 
from dotenv import load_dotenv

In [3]:
from google import genai

In [4]:
load_dotenv()

True

In [5]:
google_api_key = os.getenv("GOOGLE_API_KEY")


In [6]:
client = genai.Client(api_key = google_api_key)

In [7]:
def attach_file(file_path):
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}. Skipping attachment.")
        return None
    print(f"Attempting to attach file: {file_path}")
    try:
        file_extension = os.path.splitext(file_path)[1].lower()

        if file_extension in ['.pdf', '.docx', '.txt']:
            uploaded_file = client.files.upload(file = file_path) 
            print(f"Uploaded file for attachment (ID: {uploaded_file.uri}): {file_path}")
            return uploaded_file            
        else:
            print(f"Warning: Unsupported file type for Gemini multimodal input: {file_path}.")
            print("Supported types documents via File API (e.g., PDF, DOCX, TXT).")
            return None 

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None 


In [8]:
def fetch_questions(file_path=None):
    RESPONSE_JSON = {
        "mcqs": [
            {
                "mcq": "multiple choice question1",
                "options": {
                    "a": "choice here1",
                    "b": "choice here2",
                    "c": "choice here3",
                    "d": "choice here4",
                },
                "correct": "correct choice option",
            },
            { 
                "mcq": "multiple choice question3",
                "options": {
                    "a": "choice here1",
                    "b": "choice here2",
                    "c": "choice here3",
                    "d": "choice here4",
                },
                "correct": "correct choice option",
            }
        ]
    }

    prompt_template = f"""
    You are an expert in generating MCQ type quiz on the basis of provided content.
    Given the above text/file, create a quiz of 2 multiple choice questions.
    Make sure the questions are not repeated and check all the questions to be conforming the text as well.
    Make sure to format your response like the RESPONSE_JSON below and use it as a guide.
    Ensure to make an array of 2 MCQs referring the following response json.
    {RESPONSE_JSON}
    """
    content_parts = [prompt_template] 

    if file_path:
        attached_content = attach_file(file_path) 
        if attached_content:
            content_parts.append(attached_content)
            print(content_parts)
        else:
            print(f"Skipping file attachment for {file_path} due to previous error/unsupported type.")

    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents= content_parts
        )
        if response.text:
            cleaned_json_string = response.text.strip()
            if cleaned_json_string.startswith("```json"):
                cleaned_json_string = cleaned_json_string[len("```json"):].strip()
            if cleaned_json_string.endswith("```"):
                cleaned_json_string = cleaned_json_string[:-len("```")].strip()
            try:
                # print(response.text)
                parsed_response = json.loads(cleaned_json_string).get("mcqs",[])
                display(Markdown(f"**Bot Response (Parsed JSON):**\n```json\n{json.dumps(parsed_response, indent=2)}\n```"))
            except json.JSONDecodeError as json_err:
                print(f"Error parsing JSON response: {json_err}")
                display(Markdown(f"**Bot Response (Raw Text - JSON Parse Error):**\n```\n{response.text}\n```"))
        else:
            print("Bot: (No text generated or response is empty.)")
            if response.prompt_feedback:
                print(f"Prompt feedback: {response.prompt_feedback}")
                if response.prompt_feedback.block_reason:
                    print(f"  Block Reason: {response.prompt_feedback.block_reason}")
                if response.prompt_feedback.safety_ratings:
                    print(f"  Safety Ratings: {response.prompt_feedback.safety_ratings}")

    except Exception as e:
        print(f"An error occurred during content generation: {e}")
    return parsed_response

In [9]:
questions = fetch_questions("The Art of Capacity Planning_ Scaling Web Resources in the Cloud.pdf")

Attempting to attach file: The Art of Capacity Planning_ Scaling Web Resources in the Cloud.pdf
Uploaded file for attachment (ID: https://generativelanguage.googleapis.com/v1beta/files/voa503oz69o4): The Art of Capacity Planning_ Scaling Web Resources in the Cloud.pdf
["\n    You are an expert in generating MCQ type quiz on the basis of provided content.\n    Given the above text/file, create a quiz of 2 multiple choice questions.\n    Make sure the questions are not repeated and check all the questions to be conforming the text as well.\n    Make sure to format your response like the RESPONSE_JSON below and use it as a guide.\n    Ensure to make an array of 2 MCQs referring the following response json.\n    {'mcqs': [{'mcq': 'multiple choice question1', 'options': {'a': 'choice here1', 'b': 'choice here2', 'c': 'choice here3', 'd': 'choice here4'}, 'correct': 'correct choice option'}, {'mcq': 'multiple choice question3', 'options': {'a': 'choice here1', 'b': 'choice here2', 'c': 'choi

**Bot Response (Parsed JSON):**
```json
[
  {
    "mcq": "Who are the authors of the book 'The Art of Capacity Planning: Scaling Web Resources'?",
    "options": {
      "a": "John O'Reilly & Arun Smith",
      "b": "Arun Kejariwal & John Allspaw",
      "c": "Brian Anderson & Nicholas Adams",
      "d": "Rebecca Demarest & Karen Montgomery"
    },
    "correct": "b"
  },
  {
    "mcq": "According to the book, what is 'diagonal scaling' in the context of capacity planning?",
    "options": {
      "a": "The process of adding more identical nodes to an existing infrastructure to distribute load.",
      "b": "The capability of adding capacity by increasing the resources internal to a single server, such as CPU, memory, disk, and network.",
      "c": "The process of vertically scaling the horizontally scaled nodes that an enterprise already has in the infrastructure.",
      "d": "The method of distributing load across geographically dispersed datacenters."
    },
    "correct": "c"
  }
]
```

In [26]:
for question in questions:
    print(question.get("mcq"))
    for i in ['a','b','c','d']:
        print(i,').', question.get("options").get(i))

What approach to capacity planning does "The Art of Capacity Planning" primarily advocate?
a ). Relying on complex mathematical models and simulations.
b ). Conducting extensive benchmarks in artificial laboratory settings.
c ). An adaptive, practical approach focused on real-world usage and tools.
d ). Strictly adhering to static theoretical models for resource allocation.
According to the preface, which of the following developments significantly contributed to the decision to write the second edition of the book?
a ). The advent of quantum computing and its impact on data processing.
b ). The widespread adoption of public cloud computing, including autoscaling and microservices.
c ). A renewed industry focus on optimizing single-server performance.
d ). A shift from web resources to desktop application scaling challenges.
